In [ ]:
#!/usr/bin/python
# Modified 11/29/2019 by HYK
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!
# activation cascade with three components
# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen
"""
import roadrunner
import numpy as np
from numpy import random as rand
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function
def doSimulation(tc,nc,beta_x,gamma_x):
    r = loada("""
    # define cell cycling parameters
    TCycle = 10                                    
    V := (time/TCycle + 1)
    
    #define ode    
    R1: => Px ; (kX*V)*(S/V)*(1-Px)      # promoter activation
    R2: Px => ; lX*(Y/V)/(Y/V+KY/V)*Px    # promoter inactivation of X
    R3: => X ; (alphaX*V)*Px           # synthesis of X, rate increases with 
    R4: X => ; X*deltaX                # degradation of X
    R5: => Py ; (kY*V)*(S/V)*(1-Py)*(X/V)/(KX/V+X/V)     # activation of pY by X
    R6: => Y ; (alphaY*V)*Py
    R7: Y => ; Y*deltaY

   
   #define initial values
    S = 1            # the concentration of signal * (normalized units) 
    Px = 0             # initial promoter state of X in the state (inactive)  
    X = 0              # the number of copies of X
    Py = 0              # the promoter state of Y
    Y = 0              # the number of copies of Y

    kX = 0.02
    KX = 30
    lX = 20
    alphaX = 1300          # synthesis rate of X (copies/hr)
    deltaX = 1             # degradation rate (1/hr)
    kY = 0.02
    KY = 30               # silencing rate
    alphaY = 1000.0         # synthesis rate of Y (copies/hr)
    deltaY = 1             # degradation rate (1/hr)
    """)
    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time 
    
    # redefine parameters here, if necessary
    r.lX = beta_x
    r.alphaY = gamma_x
    ############################
    dt = 0.1;     # time interval for plotting
    result=[]
    for i in range(0,nc,1):       # for loop over the total number of cell cycles
        if i == 0:
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','Px','X','Py','Y']) 
            for row in SubResult:
                result.append(row)
        else:
            r.X = rand.binomial(r.X, 0.5)   # randomly partition X due to cell division
            r.Y = rand.binomial(r.Y, 0.5)   # randomly partition Y due to cell division
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','Px','X','Py','Y'])   # automatic update of activation state?            
            for row in SubResult:
                row[0]=row[0]+(i*tc)     # update the time vector
                result.append(row)
    result = ar(result)
    return result

#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)
        
#generate fractional overlap
def generatef12(repnum,beta_x,gamma_x,threshold):
    
    F12 = []; # fractional overlap
    T12 = []; # overlap life time
    Yc   = [];# steady state [Y]
    for rep in range(repnum):
        #print('rep: {}'.format(rep))
        df = pd.read_csv('/external/Sam/JRSI/NegativeFeedBack/data/data_{}_b_{}_g_{}.csv'.format(rep,np.round(beta_x,2),np.round(gamma_x,2)),header = None)
        X = df.iloc[:,2]
        Y = df.iloc[:,4]
        x_ind = X[X>thresh].index
        y_ind = Y[Y>thresh].index
        y_ind_zero = Y[Y>0].index
        if len(x_ind) == 0:
            continue
        if len(y_ind_zero) == 0: # if Y doesn't turn on then steady state [Y] = 0
            Yavg = 0
        else:
            Yavg = np.mean(Y[y_ind_zero]) # average [Y] while it's activated to get steady state [Y]
        F12.append(float(len(x_ind & y_ind))/len(x_ind))
        T12.append(len(x_ind & y_ind)*0.1) # every time step is 0.1 hrs
        Yc.append(Yavg)
    
    return np.mean(F12),np.mean(T12),np.mean(Yc)
        
#run a series of simulation replicates for a fixed parameter set
def runsimset(repnum,beta_x,gamma_x):
    for i in range(0,repnum,1):
        result = doSimulation(10,40,beta_x,gamma_x)
        beta_x_log = np.log10(beta_x)
        gamma_x_log = np.log10(gamma_x)
        print(beta_x_log)
        print(gamma_x_log)
        #file format: t V PX X PY Y
        writeCSV('/external/Sam/JRSI/NegativeFeedBack/data/data_{}_b_{}_g_{}.csv'.format(i,np.round(beta_x_log,2),np.round(gamma_x_log,2)),result)

In [ ]:
#### MAIN SCRIPT #######
print(' run the simulation over a range of beta_x and gamma_x parameters...')
b_range_log = np.arange(0.0,2.0,0.2)
g_range_log = np.arange(2.0,4.0,0.2)


b_range = 10**b_range_log
g_range = 10**g_range_log
for b in b_range:
    for g in g_range:
        runsimset(500,float(b),float(g))


In [ ]:
b_range = np.arange(0.0,2.0,0.2) #can't let silencing rate constant go to zero
g_range = np.arange(2.0,4.0,0.2)
thresh = 0
print(' populate the fractional overlap value in a matrix with row = b_range and col = g_range...')
Fr = np.zeros([len(b_range),len(g_range)])
Tr = np.zeros([len(b_range),len(g_range)])
Yr = np.zeros([len(b_range),len(g_range)])
for b_ind in range(len(b_range)):
    for g_ind in range(len(g_range)):
        #print('b: {}'.format(b_ind))
        #print('g: {}'.format(g_ind))
        Fr[b_ind,g_ind],Tr[b_ind,g_ind],Yr[b_ind,g_ind] = generatef12(500,b_range[b_ind],g_range[g_ind],thresh)


In [ ]:
print(' plot this as a heatmap..')
Fr_df = pd.DataFrame(Fr,columns = g_range,index = b_range)
Tr_df = pd.DataFrame(Tr,columns = g_range,index = b_range)
Yr_df = pd.DataFrame(Yr,columns = g_range,index = b_range)
ax = sns.heatmap(Fr_df)
ax.invert_yaxis()
plt.show()
ax = sns.heatmap(Tr_df)
ax.invert_yaxis()

plt.savefig("test.svg")
plt.show()
#ax = sns.heatmap(Yr_df,vmin = thresh,vmax = thresh+1)
#ax.invert_yaxis()
#plt.show()